In [ ]:
cd /p/home/jusers/todorov3/jusuf/bmp_code

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin
from datetime import datetime
import pingouin as pg
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from pingouin import ttest

import pingouin as pg
from joblib import Parallel, delayed
from itertools import product
import warnings

from bmp_config import path_data, envcode2env
from bmp_behav_proc import *
load = 1
if load:
    fnf_fhl = pjoin(path_data,'dfcs_fixhistlen.pkl')
    print(fnf_fhl)
    print( str(datetime.fromtimestamp(os.stat(fnf_fhl).st_mtime)))
    dfcs_fixhistlen = pd.read_pickle(fnf_fhl )
else:
    fnf = pjoin(path_data,'df_all_multi_tsz__.pkl.zip')
    print(fnf)
    print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
    df_all_multi_tsz = pd.read_pickle(fnf)
    df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                            ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
    df,dfall,ES_thr,envv,pert = addBehavCols2(df);
    dfc = df.copy()
    dfcs,dfcs_fixhistlen,dfcs_fixhistlen_untrunc,histlens  = addWindowStatCols(dfc, ES_thr, 
                                        varn0s = ['error','error_pscadj','error_pscadj_abs'])
    dfcs_fixhistlen, me_pct_excl = truncLargeStats(dfcs_fixhistlen_untrunc, histlens, 5.)
import pingouin as pg
dfcs_fixhistlen['environment'] = dfcs_fixhistlen['environment'].astype(int)
df_ = dfcs_fixhistlen[['environment','subject','trials','error_pscadj_abs_Tan29']]
assert not df_.duplicated(['subject','trials']).any()
all_suffixes = 'mav,std,invstd,mavsq,mav_d_std,mav_d_var,Tan,invmavsq,invmav,std_d_mav,invTan'.split(',')

dfcs_fixhistlen0 = dfcs_fixhistlen.copy()
print(len(dfcs_fixhistlen))
c = ~  np.isinf(dfcs_fixhistlen['err_sens']) 
dfcs_fixhistlen = dfcs_fixhistlen.loc[c]
print(len(dfcs_fixhistlen))
c2 = dfcs_fixhistlen['err_sens'] > -41
dfcs_fixhistlen = dfcs_fixhistlen.loc[c2]
print(len(dfcs_fixhistlen))

# make sure length n  is inside
varn0 = 'error_pscadj'
n = 3
for suffix in all_suffixes:
    s = f'{varn0}_{suffix}{n}'
    if s not in dfcs_fixhistlen.columns:
        print(s, 'not in dfcs_fixhistlen.columns')
    assert s in dfcs_fixhistlen.columns

print('all_suffixes = ', all_suffixes )

In [ ]:
prl_path_data = pjoin(path_data, '../NIH_behav_intermed_data', 'mixedlm_prl')
fnf =  pjoin(prl_path_data, f'corrs_alltogether.pkl.zip') 

#fnf =  pjoin(prl_path_data, f'corrs_alltogether_noebm_3cond.pkl.zip') 
fnf_mesubj =  pjoin(prl_path_data, f'corrs_mesubj_alltogether.pkl.zip') 

load = 1
if load:
    cocoln2corrs_per_subj = pd.read_pickle(fnf)
    print(str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
    cocoln2corrs_mesubj = pd.read_pickle(fnf_mesubj)
else:
    print('Not loaded, need recalc')

# Recalc

In [ ]:
min_range = 3
max_range = 40
N = 41
n_jobs = 256


# max_range = 21
# N = 21
#n_jobs = 1
# min_range = 20
# max_range = 25

In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
from sklearn.model_selection import cross_val_score, train_test_split
# if use shepherd, it is rather slow even in parallel mode
#corr_methods = ['spearman','shepherd']
corr_methods = ['spearman', 'pearson']

#corr_methods = [ 'pearson']
#cocols = ['None', 'env', 'ps2_']  # maybe I don't need ps2_ actually
cocols = [ 'env']  # maybe I don't need ps2_ actually

varn0s = ['error_pscadj', 'error_pscadj_abs']
varn_suffixes = ['std', 'invstd', 'mavsq', 'mav_d_std', 'mav_d_var', 'Tan']

std_mavsz_range = list(range(min_range, min(N, max_range)) )
if N < max_range:
    std_mavsz_range += list(range(N, max_range, 3))

args = list(product(corr_methods, cocols, std_mavsz_range, varn0s, varn_suffixes))
print(len(args))

In [ ]:
# s2_ 22 error_pscadj_abs mav_d_std
# ps2_ 22 error_pscadj_abs mav_d_var
# ps2_ 23 error_pscadj std

In [ ]:
# compute corrs within subject
do_ebm = 1
n_folds = 5
#backend = 'loky'
backend = 'multiprocessing' 

def run_corr(df_big, corr_method, cocoln, std_mavsz_, varn0, varn_suffix):
    def f(df_):
        with warnings.catch_warnings(record=True) as w:
            warnings.filterwarnings('ignore',category=UserWarning)
            warnings.filterwarnings('ignore',category=RuntimeWarning)

            try:
                r = pg.corr( df_[varn], df_['err_sens'],  method=corr_method)
            except ValueError as e:
                r = pd.DataFrame({'n': [len(df_)], 'r': [np.nan], 'p-val': [np.nan]})


        r['method'] = corr_method
        return r

    def f_ebm(df_):
        X_train = df_[varn].values.reshape(-1, 1) 
        y_train = df_['err_sens'].values#.reshape(-1, 1) 
        #print(X_train.shape, y_train.shape)
        ebm = ExplainableBoostingRegressor(feature_names=[varn], random_state=42)
        cv_scores = cross_val_score(ebm, X_train, y_train, cv=n_folds, scoring='r2')
        #print(f"Cross-validation R2 scores: {cv_scores}")
        r2 = np.mean(cv_scores)
        print(f"{varn} mean CV R2 score: {r2:.4f}")
        return r2
    
    #dfcs_fixhistlen, cocoln, std_mavsz_, varn0, varn_suffix = args
    varn = f'{varn0}_{varn_suffix}{std_mavsz_}'
    #cols = ['thr','subject']
    cols = ['subject']
    if cocoln != 'None':
        cols += [cocoln]
    assert not df_big.duplicated(cols + ['trials']).any()

    df_ = df_big
    df_ = df_[~np.isnan(df_[varn])] #NaN
    df_ = df_[~np.isnan(df_[varn])]
    df_ = df_[~np.isinf(df_[varn])]
    df_ = df_[~np.isinf(-df_[varn])]
    df_ = df_[~np.isinf(df_['err_sens'])]
    df_ = df_[~np.isinf(-df_['err_sens'])]


    g = df_.groupby(cols, observed=True)
    corrs_per_subj = g.apply(f, include_groups=False)
    corrs_per_subj = corrs_per_subj.rename(columns={'p-val':'pval'})

    if do_ebm:
        corrs_per_subj['R2_ebm'] = g.apply(f_ebm, include_groups=False)

    corrs_per_subj['std_mavsz'] = std_mavsz_
    corrs_per_subj['histlen'] = std_mavsz_
    corrs_per_subj['varn'] = varn
    corrs_per_subj['varn0'] = varn0
    corrs_per_subj['cocoln'] = cocoln
    corrs_per_subj['varn_suffix'] = varn_suffix

    print(corr_method, cocoln, std_mavsz_, varn0, varn_suffix)
    return corrs_per_subj.reset_index()

ind = 0
if n_jobs > 1:
    # Execute in parallel
    prlcorr = Parallel(n_jobs=n_jobs, backend = backend)\
        (delayed(run_corr)( dfcs_fixhistlen,*arg ) for arg in args)
    
else:
    #raise ValueError('I\'m lazy to implement')
    prlcorr = []
    for arg in args:
        ind += 1
        print(f'Running {ind}/{len(args)}')
        res = run_corr(dfcs_fixhistlen, *arg)
        prlcorr.append(res)
cocoln2corrs_per_subj = pd.concat(prlcorr).reset_index()

#coefficient of determination for the ranks if Spearman.
cocoln2corrs_per_subj.loc[cocoln2corrs_per_subj['method'] == 'spearman','R2'] =\
     cocoln2corrs_per_subj['r']**2
print('Calc finished well!')

In [ ]:
# compute corrs after average across subjects (pretty useless acutally) 
# env
def run_corr_me(df_big, corr_method, cocoln, std_mavsz_, varn0, varn_suffix):
    def f(df_):
        with warnings.catch_warnings(record=True) as w:
            warnings.filterwarnings('ignore',category=UserWarning)
            warnings.filterwarnings('ignore',category=RuntimeWarning)

            r = pg.corr( df_[varn], df_['err_sens'],  method=corr_method)
        r['method'] = corr_method
        return r
    
    varn = f'{varn0}_{varn_suffix}{std_mavsz_}'
    cols_av = ['trials']
    cols = []
    if cocoln != 'None':
        cols += [cocoln]
    cols_keep = cols_av + cols + ['err_sens',varn]

    df_ = df_big
    df_ = df_[~np.isnan(df_[varn])] #NaN
    df_ = df_[~np.isinf(df_[varn])]
    df_ = df_[~np.isinf(-df_[varn])]
    df_ = df_[~np.isinf(df_['err_sens'])]
    df_ = df_[~np.isinf(-df_['err_sens'])]

    dfcs_fixhistlen_me = df_[cols_keep].\
        groupby(cols_av + cols, observed=True).mean().reset_index()
    assert not dfcs_fixhistlen_me.duplicated(cols + ['trials']).any()                


    if len(cols) == 0:
        corrs_mesubj = f(dfcs_fixhistlen_me)
    else:
        corrs_mesubj = dfcs_fixhistlen_me.groupby(cols, 
            observed=True).apply(f, include_groups=False)
    corrs_mesubj = corrs_mesubj.rename(columns={'p-val':'pval'})
    corrs_mesubj['varn'] = varn
    corrs_mesubj['varn0'] = varn0
    corrs_mesubj['cocoln'] = cocoln
    corrs_mesubj['std_mavsz'] = std_mavsz_
    corrs_mesubj['histlen'] = std_mavsz_
    corrs_mesubj['varn_suffix'] = varn_suffix

    print(cocoln, std_mavsz_, varn0, varn_suffix)
    return corrs_mesubj.reset_index()

ind = 0
if n_jobs > 1:
    # Execute in parallel
    #backend = 'multiprocessing' # 'loky'
    backend = 'multiprocessing' 
    prlcorr = Parallel(n_jobs=n_jobs, backend = backend)\
        (delayed(run_corr_me)( dfcs_fixhistlen,*arg ) for arg in args)

else:
    #raise ValueError('I\'m lazy to implement')
    prlcorr = []
    for arg in args:
        ind += 1
        print(f'Running {ind}/{len(args)}')
        res = run_corr_me(dfcs_fixhistlen, *arg)
        prlcorr.append(res)
cocoln2corrs_mesubj = pd.concat(prlcorr).reset_index()
print('Calc finished well!')

In [ ]:
import gc; gc.collect()

In [ ]:
cocoln2corrs_per_subj.to_pickle(fnf)
cocoln2corrs_mesubj.to_pickle(fnf_mesubj)

# Analyze: 

## Corr for all without regard for env/ps2_ (pooled trials, not used now)

###  all trials together, mean across subjects (useless actually)

In [ ]:
ttrs_pos = cocoln2corrs_mesubj.query('cocoln == "None" and pval <= 0.05 and r > 0')
varnames = list(ttrs_pos.reset_index()['varn'].values)
print(varnames )
print('For mean,then corr {} varnames are >0 correlated'.format(len(varnames) ) )
ttrs_pos[['varn','pval','r','power']]

### all trial together, per subj 

In [ ]:
# want statistically significant correlations (so look at sets per subject)
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r', cols_addstat=['r'])
    return ttrs
m2vns={}

# include groups is whether df that f receives as arg has grouping columns or not
g = cocoln2corrs_per_subj.query('cocoln == "None"').\
    groupby(['varn','varn0','varn_suffix','method','histlen'], observed=True)
print(g.size().max())
ttrs = g.apply(f, include_groups=False)

#display(ttrs.query('pval <= 0.05 and ttstr == "r < 0"')\
#        [['pval','ttstr']])

ttrs_pos = ttrs.query('histlen > 3 and pval <= 0.05 and ttstr == "r > 0"')
#display(ttrs_pos[['pval','ttstr']])
for method in cocoln2corrs_per_subj.method.unique():
    varnames = ttrs_pos.loc[ttrs_pos.index.get_level_values('method') == method].\
        reset_index()['varn'].values
    varnames = list(varnames)
    #varnames = list(ttrs_pos[(None,method)])
    #print(varnames )
    print(method, len(varnames), ' varnames are >0 correlated' )
    m2vns[method] = varnames

#print( ', '.join(varnames) )
#print ('Dif between methods ',set(m2vns['spearman']) ^ set(m2vns['shepherd']) )

In [ ]:
ttrs_pos.columns

In [ ]:
# Romain goes maybe in sept 2026 to Japan

In [ ]:
histlens_sigpos = ttrs_pos.reset_index().groupby(['method','varn0','varn_suffix'])['histlen'].unique().to_frame()
#histlens_sigpos
for irow,row in histlens_sigpos.iterrows():
    m, varn0, varn_suffix = row.name
    histlens = row['histlen']
    from bmp_base import find_continuous_streaks
    streaks = find_continuous_streaks(sorted(histlens))
    #m = row.menthod
    #hls = ",".join(map(str,sorted(histlens)))
    print(f'{m}: {varn0}_{varn_suffix:9}, histlens={streaks}')
    # if len(histlens) == 1:
    #     print(f'Only one histlen {histlens[0]}')
    # else:
    #     print(f'Multiple histlens {histlens}')

In [ ]:
#ttrs_pos = ttrs_pos.query('method == "spearman"')
ttrs_pos = ttrs.query('histlen > 3 and pval <= 0.05 and ttstr == "r > 0"')
ttrs_pos = ttrs_pos.query('method == "pearson"')
ttrs_pos = ttrs_pos.sort_values(['pval'])
ttrs_pos[['pval','power','r_mean','r_std','T']]

In [ ]:
ttrs_pos = ttrs_pos.query('method == "spearman"')
ttrs_pos = ttrs.query('histlen > 3 and pval <= 0.05 and ttstr == "r > 0"')
#ttrs_pos = ttrs_pos.query('method == "pearson"')
ttrs_pos = ttrs_pos.sort_values(['pval'])
ttrs_pos[['pval','power','r_mean','r_std','T']]

In [ ]:
print(ttrs_pos.iloc[1])

In [ ]:
ttrs_pos_ = ttrs_pos.reset_index().\
    query('method == "spearman" and varn_suffix == "Tan" and varn0 == "error_pscadj_abs"')

display(ttrs_pos_[['histlen','pval','power','r_mean','r_std']])
print(find_continuous_streaks(ttrs_pos_['histlen'].values))
print(ttrs_pos_['histlen'].values)

In [ ]:
ttrs_pos = ttrs_pos.query('varn.str.contains("_invstd")').sort_values(['r_mean'], ascending=False)
ttrs_pos[['pval','T','r_mean','r_std']]

In [ ]:
#ttrs_pos.query('varn.str.endswith("std2")')

In [ ]:
#df_ = pd.DataFrame( {'varn':m2vns['spearman'] } )
#df_['']

# std2
# invstd 4-39
# abs invstd 2-39
# abs mav d var 4-39

m2vns['spearman']

In [ ]:
# plot ES vs windows stats
import warnings
varnames = list(ttrs_pos.reset_index()['varn'].values)
if len(varnames) <= 10:
    print(varnames)
    for env in ['stable','random']:
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore',category=FutureWarning)
            me = dfcs_fixhistlen.query('env == @env').\
                groupby(['subject','env'])[['err_sens']+varnames].mean().reset_index()
            fg = sns.pairplot(data=me, 
                     y_vars = ['err_sens'], hue='env',
                         x_vars=['err_sens'] + varnames)#,corner=1)

    me = dfcs_fixhistlen.\
        groupby(['subject'])[['err_sens']+varnames].mean().reset_index()
    fg = sns.pairplot(data=me, 
             y_vars = ['err_sens'], 
                 x_vars=['err_sens'] + varnames)#,corner=1)
else:
    print('Too long list of varnames ')

In [ ]:
gc.collect()

## Corr within env (of ES and windows-stat measues)


### mesubj (useless)

In [ ]:
# it gives 0 because for random it does not make sense to average across participants
# because randomness is not consistent across them
env2varnames = {}
ttrs = cocoln2corrs_mesubj['env']
for env in ['stable','random']:
    ttrs_pos = ttrs.query('pval <= 0.05 and r > 0 and'
                         ' env == @env')
#display(ttrs_pos[['pval','ttstr']])
    varnames = list(ttrs_pos.reset_index()['varn'].values)
    print(env,len(varnames))
    env2varnames[env] = varnames
varnames_isec = set(env2varnames['stable']) & set(env2varnames['random'])
varnames_isec = list(varnames_isec)
print('random ', set(env2varnames['random']))
print('isec varnames',varnames_isec  )

print('For mean, stable&random corr {} varnames are >0 correlated'.format(len(varnames) ) )

### within subject

In [ ]:
# Find with windows stats are positive bot for random AND for stable
#from behav_proc import compare0
def f(df):
    if len(df) == 0:
        return None
    ttrs_ = compare0(df, 'r')
    return ttrs_

ttrs = cocoln2corrs_per_subj.query('cocoln == "env"').\
    groupby(['method','varn','env','varn0','varn_suffix','histlen'], observed=True).apply(f)
print('Calc finished')

In [ ]:
# for Tan 20
#coln_to_analyze = 'error_pscadj_abs_Tan20'
coln_to_analyze = 'error_pscadj_Tan20'
ttrs.query('varn == @coln_to_analyze')#.sort_values(['pval'])

In [ ]:
print('All positive, both envs (separately)')
best = {}
for method in cocoln2corrs_per_subj.method.unique():
    #print(getAddInfo())
    #display(ttrs.query('pval <= 0.05 and thr in @thrs')\
    #        [['pval','ttstr']])

    ttrs_pos = ttrs.query('method == @method and pval <= 0.05 and ttstr == "r > 0"')
    #display(ttrs_pos[['pval','ttstr']])

    env2varnames = {}
    for env in ['stable','random']:
        ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0" and'
                             ' env == @env').sort_values(['pval']).reset_index()
        print(method, env) 
        bestrow = ttrs_pos.iloc[0]
        best[(method,env)] = bestrow
        display('best row = ',bestrow) 
        varn_ = bestrow.varn
        persubj = cocoln2corrs_per_subj.query('method == @method and env == @env and varn == @varn_')
        if method == 'spearman':
            display(persubj[['subject','R2','R2_ebm']].describe(percentiles=[ .5]) )
        elif method == 'pearson':
            display(persubj[['subject','r','R2_ebm']].describe(percentiles=[ .5]) )

    #display(ttrs_pos[['pval','ttstr']])
        varnames = list(ttrs_pos['varn'].values)
        env2varnames[env] = varnames
    varnames_isec = set(env2varnames['stable']) & set(env2varnames['random'])
    varnames_isec = list(varnames_isec)
    print(method, 'isec varnames',varnames_isec  )
#print(varnames)


### EBM

In [ ]:
bestrow = best[('spearman','stable')]
print(bestrow)

from bmp_base import calc_EBM
ebm,cv_scores,r2 = calc_EBM(dfcs_fixhistlen.query('subject_ind == 1'), bestrow['histlen'], 
                            bestrow['varn0'], bestrow['varn_suffix'])

In [ ]:
%matplotlib inline
from interpret import show
global_expl = ebm.explain_global()
print(cv_scores,r2)
show(global_expl)
plt.show()

In [ ]:
env2varnames

In [ ]:
# To be put into paper
mer = cocoln2corrs_per_subj.query('cocoln == "env"').\
    groupby(['varn','env'], observed=True)['r'].mean()

#print(getAddInfo(),'\n')
r = ttrs.query('pval <= 0.05 and ttstr == "r > 0" '
    ' and varn.isin(@varnames_isec)').reset_index()[['varn','env','pval']].set_index(['varn','env'])
#d = r.T.to_dict()
#print(d)

if r.empty:
    print('no significant correlations in stable and random separately found')
for varn in varnames_isec:
    st = r.loc[(varn,'stable'),'pval']
    ra = r.loc[(varn,'random'),'pval']
    stabr = mer[(varn,'stable')]
    randr = mer[(varn,'random')]
    
    s = (f"{varn}: For random r>0 has p-val={ra:.2e} (mean r = {randr:.3f})"
        f" and for stable r>0 has p-val={st:.2e} (mean r = {stabr:.3f})")
    print(s)

In [ ]:
# plot
h=4
varnames_toshow = varnames_isec[1:]
fg,df = relplot_multi(sep_ys_by = 'col', data=dfcs_fixhistlen, x='err_sens',ys=varnames_toshow, row='env',
                     height = h)
for ax in fg.axes.flatten():
    ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
    ax.set_title(ttl)

fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Pooled')
plt.tight_layout()

me = dfcs_fixhistlen.\
    groupby(['subject','env'])[['err_sens']+varnames_toshow].mean().reset_index()
fg,df = relplot_multi(sep_ys_by = 'col', data=me, x='err_sens',ys=varnames_toshow, row='env',
                     height = h)
for ax in fg.axes.flatten():
    ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
    ax.set_title(ttl)
fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Mean within subject')
plt.tight_layout()

In [ ]:
# fg = sns.relplot(data=dfcs_fixhistlen, col='subject',
#     y = 'err_sens', x=varnames_isec[0], row='env')

In [ ]:
from behav_proc import formatRecentStatVarnames
isec_nice = formatRecentStatVarnames(varnames_isec)
display(isec_nice)
print('; '.join(isec_nice))

In [ ]:
isec = set(env2varnames['stable']) & set(env2varnames['random'])
if len(isec):
    print('If we fix the history length first, then compute correlation '
      'within participant (separately for both environments)'
      ' and then choose variables for which r>0 stat significantly in both environments'
      ' simultaneously, we get ',', '.join(list(isec) ) )
else:
    print('We get nothing signif postive in both separately')

In [ ]:
#ttrs.query('varn == "error_pscadj_invstd10"')

In [ ]:
# plot
varnames = list(ttrs_pos.reset_index()['varn'].values)
print(varnames)
for env in ['stable','random']:
    plt.figure()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=FutureWarning)
        fg = sns.pairplot(data=dfcs.query('env == @env').\
            groupby(['subject','env'])[['err_sens']+env2varnames[env]].mean().reset_index(), 
            y_vars = ['err_sens'], hue='env',
            x_vars=['err_sens'] + env2varnames[env])#,corner=1)

In [ ]:
# You are an expert in python pandas and data science.
# I have a pandas dataframe with the following columns
# ttrs_pos.columns = ['alt', 'val1', 'ttstr','varn','env']
# 'ttstr' can take two possible string values '>0' and '<0'. 
# 'env' can take two string values 'stable' and 'random'. 
# 'varn' can take several possible string values.
# I want to select rows that have ttstr = '>0' for both stable and random values of 'env'.
# How to do it?

## Corr within ps2_ (of ES and windows-stat measues)

In [ ]:
# PS_
# all corrs are positive within participants
from behav_proc import compare0
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r')
    return ttrs
ttrs = cocoln2corrs_per_subj.query('cocoln == "ps2_"').\
    groupby(['varn','ps2_'], observed=True).apply(f)
print(getAddInfo())
ttrssig = ttrs.query('pval <= 0.05')
#display(ttrssig[['pval','ttstr','T']])

ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0"')
display(ttrs_pos[['pval','ttstr']])
varnames = list(ttrs_pos.reset_index()['varn'].values)
print(varnames)
#loc[:,'error_pscadj_std5',:]

# ttrs = corrs_per_subj2.\
#     groupby(['thr','ps_']).apply(f)
# print(getAddInfo())
# display(ttrs.query('pval <= 0.05 and thr in @thrs'))

In [ ]:
ps_v = dfall.reset_index()['ps2_'].unique()
len(ps_v), ps_v

In [ ]:
# count number of significant phases
nu = ttrs_pos.reset_index().groupby(['varn'])[['ttstr','ps2_']].nunique()
nu = nu.reset_index()
display(nu[nu['ps2_'] > 2])

In [ ]:
varns_good = nu[nu['ps2_'] == len(ps_v)].varn.values
print(varns_good)

varns_pre_good = nu[nu['ps2_'] == len(ps_v)-1].varn.values
print(varns_pre_good)

from behav_proc import formatRecentStatVarnames
isec_nice = formatRecentStatVarnames(varns_good)
display(varns_good)
print('; '.join(isec_nice))

In [ ]:
ttrs_pos.query('varn.isin(@varns_pre_good)')

In [ ]:
ttrssig_g = ttrssig_pos.reset_index().query('varn in @varns_good')\
    [['varn','ps_','pval','ttstr']]
display(ttrssig_g)

In [ ]:
mer = cocoln2corrs_per_subj.query('cocoln == "ps2_"').\
    groupby(['varn','ps2_'], observed=True)['r'].mean()

for varn in varns_good:
    print('  ',varn)    
    
    r = ttrssig_g.query('varn == @varn').\
        reset_index()[['varn','ps2_','pval']]
    d = r.T.to_dict()
    #print(d)
    s = ''
    for k,v in d.items(): 
        ps_nice = ps_2nice[v['ps2_']].lower()
        fv = v['pval']
        s += f"for {ps_nice} "            
        s += f' mean r = {mer[(varn,v["ps2_"])]:.3f} and p-val={fv:.3e}; '        
            
        
    print(s)

## Histlen individ per subject

In [ ]:
# dfcs_multi
# good to add block name because we make a pause between so supposedly we loose memory about last errors
dfcs2 = dfc.sort_values(
    ['pert_seq_code', 'subject', 'trial_group_col_calc','trials'])
grp = dfcs2.\
    groupby(['pert_seq_code', 'subject', 'trial_group_col_calc','block_name'],
           observed=True)

vars_to_av = ['error_pscadj', 'error_pscadj_abs'] 

dfs = []
#for std_mavsz_ in range(2,4):
for std_mavsz_ in range(2,30):    
    print(std_mavsz_)
    df_ = dfcs2.copy()
    df_['histlen'] = std_mavsz_
    for varn in vars_to_av:
        for g,gi in grp.groups.items():
            df_.loc[gi,f'{varn}_std'] = df_.loc[gi,varn].shift(1).rolling(std_mavsz_).std()   
            df_.loc[gi,f'{varn}_mav'] = df_.loc[gi,varn].shift(1).rolling(std_mavsz_).mean()   
                
        df_[f'{varn}_invstd'] = 1/df_[f'{varn}_std']
        df_[f'{varn}_var']    = df_[f'{varn}_std'] ** 2
        df_[f'{varn}_mavsq']  = df_[f'{varn}_mav'] ** 2
        df_[f'{varn}_mav_d_std']  = df_[f'{varn}_mav'].abs() / df_[f'{varn}_std']
        df_[f'{varn}_mav_d_var']  = df_[f'{varn}_mav'].abs() / df_[f'{varn}_var']
        df_[f'{varn}_Tan']    = df_[f'{varn}_mavsq'] / df_[f'{varn}_var']
    dfs += [df_]        
dfcs0 = pd.concat(dfs, ignore_index=True)

# only chose points where prev error is not too small (to compare with ES later)
dfcs_multi  = truncateDf(dfcs0, 'err_sens', q=0.0, infnan_handling='discard', 
                   cols_uniqify = ['subject','histlen']) #'env','thr',

gc.collect()

In [ ]:
# compute corrs
from behav_proc import compare0
# env
# here dfcs_multi varnames don't contain histlen, it is a sep column
histlens = dfcs_multi['histlen'].unique()
dfs = []
import pingouin as pg
for cocoln in ['None', 'env']:#,'ps_']:
    #for std_mavsz_ in histlens:    
    for varn0 in ['error_pscadj','error_pscadj_abs']:
        for varn in  [f'{varn0}_std',
                      f'{varn0}_invstd',                     
                     f'{varn0}_mavsq',
                      f'{varn0}_mav_d_std',
                      f'{varn0}_mav_d_var',
                     f'{varn0}_Tan']:            
            def f(df_):
                r = pg.corr( df_[varn], df_['err_sens'],  method='spearman')
                r['method'] = 'spearman'
                return r
            cols = ['histlen','subject']
            if cocoln != 'None':
                cols += [cocoln]
            print('Starting ' ,cocoln, varn, cols)
            assert not dfcs_multi.duplicated(cols + ['trials']).any()
            corrs_per_subj = dfcs_multi.groupby(cols, observed=True).apply(f)
            corrs_per_subj = corrs_per_subj.rename(columns={'p-val':'pval'})            
            corrs_per_subj['varn'] = varn                    
            corrs_per_subj['cocoln'] = cocoln #corrs_per_subj 
            dfs += [corrs_per_subj.reset_index()]
    
cocoln2corrs_per_subj_multi = pd.concat(dfs, ignore_index = True)

gc.collect()

In [ ]:
len(cocoln2corrs_per_subj_multi),len(cocoln2corrs_per_subj_.query('cocoln.isin(["None","env"])'))

In [ ]:
from behav_proc import formatRecentStatVarnames
def printNice(ttrs):
    for rowi,row in ttrs.iterrows():
        varn = row['varn']
        varn_nice = formatRecentStatVarnames([varn],'')[0]
        s = varn_nice
        s += f" (history length mean={row['histlen_mean']:.1f} trials, std={row['histlen_std']:.1f} trials)"
        print(s)

In [ ]:
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r', cols_addstat = ['histlen'])
    return ttrs

for cocoln in ['None', 'env']: #,'ps_']:
    # within env or not?
    print('###### cocoln = ',cocoln)
    
    # choose best p-value among different histlens
    if cocoln != 'None':
        cocoln_add = [cocoln]
    else:
        cocoln_add = []    
    df_ = cocoln2corrs_per_subj_multi.query('cocoln == @cocoln')
    cols0 = ['subject','varn']    
    cols0 += cocoln_add
    grp = df_.groupby(cols0, observed=True )
    
    # take with with the lower p-value among all histlens
    dfr = aggRows(df_, 'pval', 'min',  grp, coltake = 'corresp') 
    
    display('r info:', dfr.\
        groupby(cocoln_add + ['varn'], observed=True)['r'].describe() )
    display('(for r>0) histlen info:', 
        dfr.query('r > 0').groupby(cocoln_add + ['varn'], 
                                   observed=True)['histlen'].describe() )
    display('(for r>0) pval info:', 
        dfr.query('r > 0').groupby(cocoln_add + ['varn'], 
                                   observed=True)['pval'].describe() )
    # compare r with zero
    cols1 = ['varn']
    cols1 += cocoln_add
    ttrs = dfr.\
        groupby(cols1, observed=True).apply(f).reset_index()
    print(getAddInfo())
    
    cols = ['varn','pval','ttstr','histlen_mean','histlen_std']
    cols = cocoln_add + cols
    
    ttrs_neg= ttrs.query('pval <= 0.05 and ttstr == "r < 0"')        
    ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0"')   
    display(ttrs_neg[cols])
    display(ttrs_pos[cols])
    
    varnames = list(ttrs_pos.reset_index()['varn'].values)    
    if cocoln != 'None':
        varnames_ext = zip(varnames, list(ttrs_pos.reset_index()[cocoln].values) )
        varnames_ext  = list(varnames_ext)
        print('r>0 varnames ',varnames_ext)
    else:
        print('r>0 varnames' , ', '.join(varnames) )
    
    printNice(ttrs_pos)    